In [1]:
import os

In [2]:
from dotenv import load_dotenv
load_dotenv()

# Access the API key (this will not expose the key in your notebook)
google_api_key = os.getenv("GOOGLE_API_KEY")

# Verify it loaded correctly without exposing the key
if google_api_key:
    print("Gemini API key loaded successfully!")
else:
    print("Failed to load Gemini API key from .env file!")

Gemini API key loaded successfully!


In [4]:
import requests
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.tools import tool, InjectedToolArg
from langchain_core.messages import HumanMessage
from typing import Annotated


In [5]:
# tool create

@tool
def multiply(a:int, b:int):
    """Given 2 numbers a & b this tool returns their product"""
    return a * b

In [6]:
print(multiply.invoke({'a':5,'b':9}))

45


In [7]:
multiply.name

'multiply'

In [8]:
multiply.description

'Given 2 numbers a & b this tool returns their product'

In [9]:
multiply.args

{'a': {'title': 'A', 'type': 'integer'},
 'b': {'title': 'B', 'type': 'integer'}}

In [13]:
# tool binding

llm = ChatGoogleGenerativeAI(google_api_key=google_api_key, model="gemini-1.5-pro")
llm_with_tools = llm.bind_tools([multiply])

In [14]:
llm_with_tools.invoke("Hi! How are you ?")

AIMessage(content="I'm doing well, thank you for asking! How can I help you today?", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-1.5-pro-002', 'safety_ratings': []}, id='run-dd7f36b7-ccbc-4bd7-be28-ea93b09aa605-0', usage_metadata={'input_tokens': 23, 'output_tokens': 19, 'total_tokens': 42, 'input_token_details': {'cache_read': 0}})

In [16]:
llm_with_tools.invoke('Can you multiply 8 with 89').tool_calls

[{'name': 'multiply',
  'args': {'a': 8.0, 'b': 89.0},
  'id': '2eba831c-264f-4782-88c6-2205ab17a3f7',
  'type': 'tool_call'}]